In [3]:
# adapted from https://github.com/MateLabs/All-Conv-Keras/blob/master/allconv.py

from __future__ import print_function
import tensorflow as tf
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dropout, Activation, Conv2D, GlobalAveragePooling2D, merge
from keras.utils import np_utils
from keras.optimizers import SGD
from keras import backend as K
from keras.models import Model
from keras.layers.core import Lambda
from keras.callbacks import ModelCheckpoint
import pandas
import cv2
import numpy as np


batch_size = 32
nb_classes = 10
nb_epoch = 30

rows, cols = 32, 32

channels = 3

(X_train, y_train), (X_test, y_test) = cifar10.load_data()
print('X_train shape:', X_train.shape)
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')

print (X_train.shape[1:])

Y_train = np_utils.to_categorical(y_train, nb_classes)
Y_test = np_utils.to_categorical(y_test, nb_classes)

model = Sequential()

model.add(Conv2D(96, (3, 3), padding = 'same', input_shape=(32, 32, 3)))
model.add(Activation('relu'))
model.add(Conv2D(96, (3, 3),padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(96, (3, 3), padding='same', strides=(2,2)))
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Conv2D(192, (3, 3), padding = 'same'))
model.add(Activation('relu'))
model.add(Conv2D(192, (3, 3),padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(192, (3, 3),padding='same', strides=(2,2)))
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Conv2D(192, (3, 3), padding = 'same'))
model.add(Activation('relu'))
model.add(Conv2D(192, (1, 1),padding='valid'))
model.add(Activation('relu'))
model.add(Conv2D(10, (1, 1), padding='valid'))



model.add(GlobalAveragePooling2D())
model.add(Activation('softmax'))
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])


print (model.summary())

X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255

datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False) 

datagen.fit(X_train)
filepath="weights.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=False, mode='max')

callbacks_list = [checkpoint]

# Fit the model on the batches generated by datagen.flow().
history_callback = model.fit_generator(datagen.flow(X_train, Y_train,
                                     batch_size=batch_size),
                        steps_per_epoch=X_train.shape[0]//batch_size,
                        epochs=nb_epoch, validation_data=(X_test, Y_test), callbacks=callbacks_list, verbose=1)

pandas.DataFrame(history_callback.history).to_csv("history.csv")

model.save('keras_allconv.h5')

X_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples
(32, 32, 3)
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_19 (Conv2D)           (None, 32, 32, 96)        2688      
_________________________________________________________________
activation_17 (Activation)   (None, 32, 32, 96)        0         
_________________________________________________________________
conv2d_20 (Conv2D)           (None, 32, 32, 96)        83040     
_________________________________________________________________
activation_18 (Activation)   (None, 32, 32, 96)        0         
_________________________________________________________________
conv2d_21 (Conv2D)           (None, 16, 16, 96)        83040     
_________________________________________________________________
activation_19 (Activation)   (None, 16, 16, 96)        0         
_________________________________________________________

1562/1562 [==============================] - 857s - loss: 1.0552 - acc: 0.6294 - val_loss: 0.9419 - val_acc: 0.6865
Epoch 19/30
1562/1562 [==============================] - 858s - loss: 1.0115 - acc: 0.6435 - val_loss: 1.0010 - val_acc: 0.6669
Epoch 20/30
1562/1562 [==============================] - 858s - loss: 0.9775 - acc: 0.6565 - val_loss: 0.8790 - val_acc: 0.7039
Epoch 21/30
1562/1562 [==============================] - 857s - loss: 0.9482 - acc: 0.6682 - val_loss: 0.8541 - val_acc: 0.7053
Epoch 22/30
1562/1562 [==============================] - 856s - loss: 0.9190 - acc: 0.6812 - val_loss: 0.7900 - val_acc: 0.7276
Epoch 23/30
1562/1562 [==============================] - 857s - loss: 0.8888 - acc: 0.6909 - val_loss: 0.8101 - val_acc: 0.7291
Epoch 24/30
1562/1562 [==============================] - 859s - loss: 0.8621 - acc: 0.7014 - val_loss: 0.7564 - val_acc: 0.7445
Epoch 25/30
1562/1562 [==============================] - 853s - loss: 0.8459 - acc: 0.7066 - val_loss: 0.7833 - val_